In [1]:
import torch as t
from torch import nn
import numpy as np
import pandas as pd

from utils import Pipeline_lstm
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.simplefilter('ignore')
DEVICE = 'cuda' if t.cuda.is_available() else 'cpu'

In [2]:
df = pd.read_csv('./train.csv', index_col=0)
df_test = pd.read_csv('./test.csv', index_col=0)

df_coin = df.loc[df['coin_id'] == 'coin_1']

In [24]:
features = ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_8', 'feat_9', 'feat_10']
lookback=30

train_split = 0.8
split_ind = int(df_coin.shape[0] * train_split)

X_train = df_coin[features].iloc[:split_ind]
X_test = df_coin[features].iloc[split_ind:]

y_train = df_coin['fwd_ret_3d'].iloc[:split_ind]
y_test = df_coin['fwd_ret_3d'].iloc[split_ind:]

pipe = Pipeline_lstm(lookback=lookback)
train_ds = pipe.fit_transform(X_train, y_train)
val_ds = pipe.transform(X_test, y_test)

In [25]:
class ConvModel(nn.Module):
    def __init__(self, channels=5):
        super().__init__()
        self.conv = t.nn.Sequential(
            t.nn.Conv1d(channels, 64, 3, padding=1),
            t.nn.BatchNorm1d(64),
            t.nn.ReLU(),

            t.nn.Conv1d(64, 128, 3, padding=1),
            t.nn.BatchNorm1d(128),
            t.nn.ReLU(),

            t.nn.Flatten(),
            t.nn.Linear(1024, 256),
            t.nn.Linear(256, 1)
        )
        self.conv[0].bias.data.fill_(0.01)
        self.conv[0].weight.data.fill_(0.01)
        self.conv[-1].bias.data.fill_(0.01)
        self.conv[-1].weight.data.fill_(0.01)

    def forward(self, x):
        return self.conv(x)



In [29]:
from train_tools import TrainModel_NoLoader, pearson_c

model_conv = ConvModel(lookback).to(device=DEVICE)

lr = 3e-6
optimizer = t.optim.Adam(model_conv.parameters(),
                         lr=lr)
writer = SummaryWriter('./runs/1dConv/')
epochs=39

_ = TrainModel_NoLoader(model=model_conv,
                        loss_fn=pearson_c,
                        optimizer=optimizer,
                        train_ds=train_ds,
                        val_ds=val_ds,
                        epochs=epochs,
                        display_on_epoch = 1,
                        device=DEVICE,
                        writer=writer)

Train Loss: 0.87; Val Loss: 0.71: 100%|██████████| 39/39 [00:00<00:00, 240.53it/s]


In [30]:
from SeparatePredictor import Conv1dPredictor

lookback = 30

features = ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_8', 'feat_9', 'feat_10']

predictor = Conv1dPredictor(models=[ConvModel],
                            data=df,
                            params={'channels': lookback},
                            lookback=lookback,
                            device=DEVICE,
                            relevant_features=features)
predictor.prepare()

In [9]:
import json

with open('./epoch_shedule.json', 'r') as f:
    epoch_shedule = json.load(f)

In [31]:
epochs=30

train_params={
    'display_on_epoch':1
}
predictor.train(device=DEVICE,
                epochs=epochs,
                loss_fn=pearson_c,
                learning_rate=lr,
                # epoch_shedule=epoch_shedule,
                other_train_params=train_params)

TRAINING fcoin_1


Train Loss: 0.87; Val Loss: 0.83: 100%|██████████| 30/30 [00:00<00:00, 273.74it/s]


TRAINING fcoin_2


Train Loss: 0.88; Val Loss: 0.99: 100%|██████████| 30/30 [00:00<00:00, 247.14it/s]


TRAINING fcoin_3


...

Train Loss: 0.71; Val Loss: 1.01: 100%|██████████| 30/30 [00:00<00:00, 278.48it/s]


TRAINING fcoin_64


Train Loss: 0.82; Val Loss: 0.86: 100%|██████████| 30/30 [00:00<00:00, 204.50it/s]


TRAINING fcoin_65


Train Loss: 0.80; Val Loss: 1.12: 100%|██████████| 30/30 [00:00<00:00, 201.77it/s]


In [35]:
ind = 28800
y_preds = predictor.predict(df.iloc[ind:], device='cuda')
t.corrcoef(t.concat([
    t.tensor(df.iloc[ind:]['fwd_ret_3d'].values).reshape(-1,1).T,
    t.tensor(y_preds.values).reshape(-1,1).T
], dim=0)
)


tensor([[ 1.0000, -0.0017],
        [-0.0017,  1.0000]], dtype=torch.float64)

In [10]:
from datetime import datetime
y_pred = predictor.predict(df_test, DEVICE)
y_pred['Id'] = y_pred.index
y_pred['Predicted'] = y_pred[0]
y_pred.to_csv('./submission_{}.csv'.format(datetime.now().timestamp()),
              columns=['Id', 'Predicted'], index=False)

In [11]:
import datetime
datetime.datetime.now().timestamp()

1673972608.311652